# Import thư viện

In [ ]:
!pip install underthesea

In [ ]:
!pip install pyvi

In [ ]:
import pandas as pd
from nltk.tag import hmm
from nltk.probability import LaplaceProbDist
import pandas as pd
from sklearn.metrics import classification_report
from pyvi import ViTokenizer
from underthesea import word_tokenize
import warnings
warnings.filterwarnings("ignore")

# Import dữ liệu

In [ ]:
df = pd.read_csv('dataset.csv', header = None)
data = df[0].tolist()

In [ ]:
# Dữ liệu tách từ thủ công làm kết quả đối chiếu
data_tokenized_manual = df[1].tolist()
print("Một số câu trong tách từ thủ công: \n")
for sentence in data_tokenized_manual[0:3]:
    print(sentence)

Một số câu trong tách từ thủ công: 

đây là bài_tập mới
đây là một trong những bài_tập khó
nó đang làm bài_tập


# Các hàm viết sẵn

### Hàm đếm số lượng từ tách được

In [ ]:
def count_words(data):
    count = 0
    for sentence_index in range(0, len(data)):
        for word_index in range(0, len(data[sentence_index])):
            if data[sentence_index][word_index] == ' ' and data[sentence_index][word_index + 1] != ' ':
                count += 1
        count += 1
    return count

### Hàm đếm số lượng từ ghép

In [ ]:
def compounds(data):
    count = 0
    for sentence_index in range(0, len(data)):
        for word_index in range(0, len(data[sentence_index])):
            if '_' in data[sentence_index][word_index]:
                count += 1
    return count

### Hàm chuyển các câu được tách từ vào list

In [ ]:
def list_words(data):
    word = []
    list_data = []
    for sentence_index in range(0, len(data)):
        a = ''
        for word_index in range(0, len(data[sentence_index])):
            a = a + data[sentence_index][word_index]
            if data[sentence_index][word_index] == ' ' and data[sentence_index][word_index + 1] != ' ':
                word.append(a)
                a = ''
            if word_index == len(data[sentence_index]) - 1:
                word.append(a)
        list_data.append(word)
        word = []
    return list_data

### Hàm thống kê tần số xuất hiện của từ ghép

In [ ]:
def array_compounds(data):
    array_manual_compounds = []
    array = []
    for sentence_index in range(0, len(data)):
        for word_index in range(0, len(data[sentence_index])):
            if '_' in data[sentence_index][word_index]:
                array_manual_compounds.append(data[sentence_index][word_index].strip())
    my_dict = {i:array_manual_compounds.count(i) for i in array_manual_compounds}
    return array_manual_compounds, my_dict

### Hàm đếm số lượng từ tách đúng

In [ ]:
def count_correct_words(count_pred, pred_compounds, array_pred_compounds, count_manual, manual_compounds, array_manual_compounds):
  count_words_in_manual = 0
  count_words_not_in_manual = 0
  for sentence_index in range(0, len(array_pred_compounds)):
    if array_pred_compounds[sentence_index] in array_manual_compounds:
      count_words_in_manual += 1
    if array_pred_compounds[sentence_index] not in array_manual_compounds:
      count_words_not_in_manual += 1
  return (count_manual - (manual_compounds - count_words_in_manual) - count_words_not_in_manual * 2)

### Hàm tính các chỉ số đánh giá của tách từ

In [ ]:
def evaluation_params(pred, source, n_grams=3):
    pred_words = pred.split()
    source_words = source.split()
    
    tp, fp, fn= 0, 0, 0
    
    index = 0
    while index < len(pred_words):
        if pred_words[index] not in source_words[index:(index + n_grams)]: 
            if '_' in pred_words[index]: fp += 1
            del pred_words[index]
        else: index += 1
    
    index = 0
    while index < len(source_words):
        if source_words[index] not in pred_words[index:(index + n_grams)]:
            if '_' in source_words[index]: fn += 1
            del source_words[index]
        else: index += 1
    
    if len(pred_words) < len(source_words): words = pred_words
    else: words = source_words
    
    for index in range (len(words)):
        if pred_words[index] == source_words[index]:
            if '_' in pred_words[index]: tp += 1 
                    
    return tp, fp, fn

### Hàm đánh giá tách từ

In [ ]:
def tokenize_evaluation(pred, source, n_grams=3):
    total_words = 0
    pred_tp = 0
    pred_fp = 0
    pred_fn = 0
    
    pred_compounds, source_compounds = compounds(pred), compounds(source)
    count_pred, count_source = count_words(pred), count_words(source)
    list_pred, list_source = list_words(pred), list_words(source)
    array_pred_compounds, _= array_compounds(list_pred)
    array_source_compounds, _= array_compounds(list_source)
    
    
    total_true = count_correct_words(count_pred, pred_compounds, array_pred_compounds, count_source, source_compounds, array_source_compounds)
    
    
    for pred_sentence, source_sentence in zip(pred, source):
        total_words += len(source_sentence.split())
        if pred_sentence != source_sentence:
            tp, fp, fn = evaluation_params(pred_sentence, source_sentence, n_grams)
            pred_tp += tp
            pred_fp += fp
            pred_fn += fn
        else:
            for word in source_sentence.split():
                 if '_' in word: pred_tp += 1
    
    precision = pred_tp / (pred_tp + pred_fp)
    recall = pred_tp / (pred_tp + pred_fn)
    F1 = 2 * (precision * recall) / (precision + recall)
    
    return {
        'Accuracy': total_true / total_words, 
        'Precision': precision,
        'Recall': recall,
        'True Positive': pred_tp, 
        'False Positive': pred_fp,
        'False Negative': pred_fn,
        'F1-score': F1,
        'Tổng từ tách đúng': total_true
    }

### Hàm gán nhãn bằng HMM

In [ ]:
def HiddenMarkovModel(data_train, data_test, smoothingTechnique = None):
    data_pred = []
    trainer = hmm.HiddenMarkovModelTrainer()
    tagger = trainer.train_supervised(data_train, estimator = smoothingTechnique)
    for sentence in data_test:
        data = tagger.tag(sentence.split())
        data_pred.append(data)
    return data_pred

### Hàm lấy chuỗi nhãn

In [ ]:
def words_tag(data,list_tag):
    tag_sequence = []
    for sentence_index in range(0, len(data)):
        for word_index in range(0, len(data[sentence_index])):
            for tag_index in range(0, len(list_tag)):
                if list_tag[tag_index] in data[sentence_index][word_index]:
                    tag_sequence.append(list_tag[tag_index])
    return tag_sequence

### Hàm đếm số nhãn gán đúng

In [ ]:
def correctTagsCounter(tag_sequence_pred, tag_sequence_gold):
    counter = 0;
    for index in range(0, len(tag_sequence_pred)):
        if tag_sequence_pred[index] == tag_sequence_gold[index]:
            counter += 1
    return counter

# Tách từ

## Tách từ thủ công

### Kết quả

In [ ]:
count_manual = count_words(data_tokenized_manual)
print('Số lượng từ tách thủ công:')
print(count_manual)

Số lượng từ tách thủ công:
232


In [ ]:
manual_compounds = compounds(data_tokenized_manual)
print("Số lượng từ đơn:")
print(count_manual - manual_compounds, "\n")
print("Số lượng từ ghép:")
print(manual_compounds)

Số lượng từ đơn:
221 

Số lượng từ ghép:
11


In [ ]:
list_manual = list_words(data_tokenized_manual)
_, array_manual_compounds = array_compounds(list_manual)
print('Thống kê từ ghép được tách thủ công:')
print(array_manual_compounds)

Thống kê từ ghép được tách thủ công:
{'bài_tập': 10, 'bài_học': 1}


## Tách từ dùng longest matching

### Thuật toán

In [ ]:
def tokenizer (text, dict, show=False):
    input = text.split()
    words = []
    s = 0
    while True:
        e = len(input)
        while e>s:
            tmp_word = input[s:e]
            is_word = ""
            for item in tmp_word:
                is_word += item + " "
            is_word = is_word[:-1]
            e -= 1
            if is_word.lower() in dict:
                words.append(is_word)
                break
        if e >= len(input):
            break
        s = e + 1
    output=""
    for item in words:
        output += item.replace(" ", "_")
        output += " "
    output = output[:-1]
    return output

### Từ điển

In [ ]:
if __name__ == "__main__":
    dict = {"đây":0,"là":1,"bài tập":2,"mới":3,"một":4,"trong":5,"những":6,"khó":7,
          "nó":8,"đang":9,"làm":10,"toán":11,"này":12,"dễ":13,"cô":14,"ấy":15,
          "tập":16,"bài":17,"chồng":18,"thật":19,"dày":20,"hắn":21,"đống":22,"xếp":23,"học":24,
          "bài học":25,"cách":26,"dùng":27,"kéo":28,"cây":29,"sắc":30,"lắm":31,"bọn":32,
          "trẻ":33,"nhau":34,"cái":35,"bàn":36,"vững":37,"mọi":38,"người":39,"với":40,"tôi":41,
          "lấy":42,"cắt":43,"giấy":44,"anh":45,"quạt":46,"lớn":47,"đừng":48,"nhanh":49,"quá":50,
          "đặt":51,"gần":52,"mạnh":53,"hư":54,"rất":55,"sẽ":56,"học sinh":57,"sinh học":58 }
    data_tokenized_LM=[]
    for sentence in data:
        output = tokenizer(sentence,dict)
        data_tokenized_LM.append(output)

### Kết quả

In [ ]:
count_LM = count_words(data_tokenized_LM)
print('Số lượng từ tách được:')
print(count_LM)

Số lượng từ tách được:
232


In [ ]:
LM_compounds = compounds(data_tokenized_LM)
print("Số lượng từ đơn:")
print(count_LM - LM_compounds, "\n")
print("Số lượng từ ghép:")
print(LM_compounds)

Số lượng từ đơn:
221 

Số lượng từ ghép:
11


In [ ]:
list_LM = list_words(data_tokenized_LM)
_, array_LM_compounds = array_compounds(list_LM)
print('Thống kê từ ghép:')
print(array_LM_compounds)

Thống kê từ ghép:
{'bài_tập': 10, 'bài_học': 1}


## Tách từ dùng thư viện pyvi

In [ ]:
data_tokenized_pyvi=[]
for sentence in data:
    ouput = ViTokenizer.tokenize(sentence)
    data_tokenized_pyvi.append(ouput)

### Kết quả

In [ ]:
count_pyvi = count_words(data_tokenized_pyvi)
print('Số lượng từ tách được khi sử dụng thư viện Pyvi:')
print(count_pyvi)

Số lượng từ tách được khi sử dụng thư viện Pyvi:
230


In [ ]:
pyvi_compounds = compounds(data_tokenized_pyvi)
print("Số lượng từ đơn:")
print(count_pyvi - pyvi_compounds, "\n")
print("Số lượng từ ghép:")
print(pyvi_compounds)

Số lượng từ đơn:
217 

Số lượng từ ghép:
13


In [ ]:
list_pyvi = list_words(data_tokenized_pyvi)
_, array_pyvi_compounds = array_compounds(list_pyvi)
print('Danh sách từ ghép khi tách từ sử dụng thư viện pyvi:')
print(array_pyvi_compounds)

Danh sách từ ghép khi tách từ sử dụng thư viện pyvi:
{'bài_tập': 10, 'bài_học': 1, 'kéo_bàn': 1, 'quạt_cây': 1}


## Tách từ dùng thư viện underthesea

In [ ]:
data_tokenized_uts=[]
for sentence in data:
    data = word_tokenize(sentence, format="text")
    data_tokenized_uts.append(data)

### Kết quả

In [ ]:
count_uts = count_words(data_tokenized_uts)
print('Số lượng từ tách được khi sử dụng thư viện Underthesea:')
print(count_uts)

Số lượng từ tách được khi sử dụng thư viện Underthesea:
239


In [ ]:
uts_compounds = compounds(data_tokenized_uts)
print("Số lượng từ đơn:")
print(count_uts - uts_compounds, "\n")
print("Số lượng từ ghép:")
print(uts_compounds)

Số lượng từ đơn:
235 

Số lượng từ ghép:
4


In [ ]:
list_uts = list_words(data_tokenized_uts)
_, array_uts_compounds = array_compounds(list_uts)
print('Danh sách từ ghép khi tách bằng thư viện underthesea:')
print(array_uts_compounds)

Danh sách từ ghép khi tách bằng thư viện underthesea:
{'bài_tập': 3, 'bài_học': 1}


## Đánh giá tách từ

In [ ]:
longest_matching_evaluation = tokenize_evaluation(data_tokenized_LM, data_tokenized_manual)
pyvi_evaluation = tokenize_evaluation(data_tokenized_pyvi, data_tokenized_manual)
uts_evaluation = tokenize_evaluation(data_tokenized_uts, data_tokenized_manual)
pd.DataFrame(
    [longest_matching_evaluation, pyvi_evaluation, uts_evaluation], 
    index = ['Longest Matching', 'Pyvi', 'Underthesea']
).astype(object).T

,Longest Matching,Pyvi,Underthesea
Accuracy,1,0.982759,0.969828
Precision,1,0.846154,1
Recall,1,1,0.363636
True Positive,11,11,4
False Positive,0,2,0
False Negative,0,0,7
F1-score,1,0.916667,0.533333
Tổng từ tách đúng,232,228,225


# Gán nhãn

## Bô dữ liệu huấn luyện

In [ ]:
data_train = [[('đây', 'P'), ('là', 'V'), ('bài_tập', 'N'), ('mới', 'A')],
             [('đây', 'P'), ('là', 'V'), ('một', 'A'), ('trong', 'N'), ('những', 'D'), ('bài_tập', 'N'), ('khó', 'A')],
             [('nó', 'P'), ('đang', 'R'), ('làm', 'V'), ('bài_tập', 'N')],
             [('bài_tập', 'N'), ('toán', 'N'), ('này', 'P'), ('dễ', 'A')],
             [('cô', 'N'), ('ấy', 'P'), ('đang', 'R'), ('tập', 'V'), ('bài', 'N'), ('này', 'P')],
             [('chồng', 'N'), ('tập', 'N'), ('thật', 'R'), ('dày', 'A')],
             [('hắn', 'P'), ('đang', 'R'), ('chồng', 'V'), ('đống', 'N'), ('bài_tập', 'N')],
             [('chồng', 'N'), ('cô', 'N'), ('ấy', 'P'), ('đang', 'R'), ('xếp', 'V'), ('tập', 'N')],
             [('bài_tập', 'N'), ('xếp', 'V'), ('tập', 'N'), ('khó', 'A')],
             [('nó', 'P'), ('đang', 'R'), ('học', 'V'), ('bài', 'N'), ('này', 'P')],
             [('một', 'A'), ('bài_học', 'N'), ('cách', 'N'), ('dùng', 'V'), ('kéo', 'N')],
             [('cây', 'N'), ('kéo', 'N'), ('này', 'P'), ('sắc', 'A'), ('lắm', 'R')],
             [('bọn', 'N'), ('trẻ', 'N'), ('đang', 'R'), ('kéo', 'V'), ('nhau', 'N')],
             [('cái', 'N'), ('bàn', 'N'), ('này', 'P'), ('vững', 'A'), ('lắm', 'R')],
             [('mọi', 'D'), ('người', 'N'), ('đang', 'R'), ('bàn', 'V'), ('với', 'C'), ('nhau', 'N')],
             [('cái', 'N'), ('bàn', 'N'), ('với', 'C'), ('cái', 'N'), ('kéo', 'N')] ,
             [('tôi', 'P'), ('đang', 'R'), ('với', 'V'), ('lấy', 'V'), ('cái', 'N'), ('kéo', 'N')],
             [('kéo', 'N'), ('dùng', 'V'), ('cắt', 'V'), ('giấy', 'N')],
             [('bọn', 'N'), ('trẻ', 'N'), ('đang', 'R'), ('bàn', 'V'), ('cách', 'N'), ('dùng', 'V'), ('kéo', 'N')],
             [('nó', 'P'), ('đang', 'R'), ('kéo', 'V'), ('cái', 'N'), ('bàn', 'N')],
             [('đây', 'P'), ('là', 'V'), ('bài_tập', 'N'), ('cách', 'N'), ('dùng', 'V'), ('kéo', 'N')],
             [('anh', 'N'), ('ấy', 'P'), ('đang', 'R'), ('tập', 'V'), ('cách', 'N'), ('kéo', 'V'), ('bàn', 'N')],
             [('một', 'A'), ('cây', 'N'), ('quạt', 'N'), ('lớn', 'A')],
             [('đừng', 'V'), ('quạt', 'V'), ('nhanh', 'A'), ('quá', 'R')],
             [('nó', 'P'), ('đang', 'R'), ('tập', 'V'), ('quạt', 'V')],
             [('cô', 'N'), ('ấy', 'P'), ('tập', 'V'), ('cách', 'N'), ('quạt', 'V')],
             [('cái', 'N'), ('quạt', 'N'), ('đặt', 'V'), ('gần', 'A'), ('bàn', 'N')],
             [('nó', 'P'), ('với', 'V'), ('lấy', 'V'), ('cái', 'N'), ('quạt', 'N')],
             [('cây', 'N'), ('quạt', 'N'), ('với', 'C'), ('cái', 'N'), ('bàn', 'N')],
             [('kéo', 'V'), ('cái', 'N'), ('bàn', 'N'), ('gần', 'A'), ('cái', 'N'), ('quạt', 'N')],
             [('dùng', 'V'), ('kéo', 'N'), ('cắt', 'V'), ('quạt', 'N')],
             [('quạt', 'V'), ('mạnh', 'A'), ('dễ', 'A'), ('hư', 'A'), ('quạt', 'N')]]

## Bộ gold

In [ ]:
data_gold = [[('bài_tập', 'N'), ('cắt', 'V'), ('quạt', 'N'), ('với', 'C'), ('kéo', 'N')], 
             [('dùng', 'V'), ('kéo', 'N'), ('cắt', 'V'), ('tập', 'N'), ('gần', 'A'), ('bàn', 'N')], 
             [('kéo', 'V'), ('cái', 'N'), ('kéo', 'N'), ('gần', 'A'), ('bàn', 'N'), ('với', 'C'), ('quạt', 'N')], 
             [('nó', 'P'), ('với', 'V'), ('cây', 'N'), ('kéo', 'N'), ('gần', 'A'), ('tập', 'N'), ('bài_tập', 'N'), ('gần', 'A'), ('bàn', 'N'), ('với', 'C'), ('quạt', 'N')], 
             [('anh', 'N'), ('ấy', 'P'), ('đang', 'R'), ('bàn', 'V'), ('với', 'C'), ('cô', 'N'), ('ấy', 'P'), ('cách', 'N'), ('kéo', 'V'), ('cái', 'N'), ('bàn', 'N')], 
             [('đừng', 'V'), ('dùng', 'V'), ('quạt', 'N'), ('với', 'C'), ('kéo', 'N')], 
             [('tập', 'V'), ('cách', 'N'), ('học', 'V'), ('dùng', 'V'), ('kéo', 'N'), ('với', 'C'), ('quạt', 'N')], 
             [('bài_tập', 'N'), ('kéo', 'V'), ('cái', 'N'), ('bàn', 'N'), ('gần', 'A'), ('cô', 'N'), ('ấy', 'P'), ('với', 'C'), ('anh', 'N'), ('ấy', 'P'), ('quạt', 'V'), ('cây', 'N'), ('kéo', 'N'), ('gần', 'A'), ('bàn', 'N')]]

## Import bộ test sau khi được tách từ

In [ ]:
data_test = data_tokenized_LM[32:40]

In [ ]:
data_test

['bài_tập cắt quạt với kéo',
 'dùng kéo cắt tập gần bàn',
 'kéo cái kéo gần bàn với quạt',
 'nó với cây kéo gần tập bài_tập gần bàn với quạt',
 'anh ấy đang bàn với cô ấy cách kéo cái bàn',
 'đừng dùng quạt với kéo',
 'tập cách học dùng kéo với quạt',
 'bài_tập kéo cái bàn gần cô ấy với anh ấy quạt cây kéo gần bàn']

In [ ]:
# Các nhãn được sử dụng
list_tag = ['A','C','E','I','L','M','N','P','R','S','T','V','X','F']
# Chuyển chuỗi nhãn cho bộ gold
tag_sequence_gold = words_tag(data_gold, list_tag)

## HMM khi không dùng smoothing

In [ ]:
data_pred_noSmoothing = HiddenMarkovModel(data_train, data_test)
tag_sequence_noSmoothing = words_tag(data_pred_noSmoothing, list_tag)

tags_noSmoothing = correctTagsCounter(tag_sequence_noSmoothing, tag_sequence_gold)
print("Số nhãn đúng là:", tags_noSmoothing, "/", len(tag_sequence_gold), "\n")

tagsList_noSmoothing = {i:tag_sequence_noSmoothing.count(i) for i in tag_sequence_noSmoothing}
print("Tần số xuất hiện của các nhãn khi không sử dụng smoothing:")
print(tagsList_noSmoothing, "\n")

print('Kết quả đánh giá của mô hình HMM khi không sử dụng smoothing:\n')
print(classification_report(tag_sequence_gold,tag_sequence_noSmoothing))

Số nhãn đúng là: 60 / 67 

Tần số xuất hiện của các nhãn khi không sử dụng smoothing:
{'N': 33, 'V': 12, 'C': 6, 'A': 6, 'P': 9, 'R': 1} 

Kết quả đánh giá của mô hình HMM khi không sử dụng smoothing:

              precision    recall  f1-score   support

           A       1.00      1.00      1.00         6
           C       1.00      0.86      0.92         7
           N       0.94      0.91      0.93        34
           P       0.56      1.00      0.71         5
           R       1.00      1.00      1.00         1
           V       0.92      0.79      0.85        14

    accuracy                           0.90        67
   macro avg       0.90      0.93      0.90        67
weighted avg       0.92      0.90      0.90        67



## HMM khi dùng Laplace smoothing

In [ ]:
data_pred_laplaceSmoothing = HiddenMarkovModel(data_train, data_test, LaplaceProbDist)
tag_sequence_laplaceSmoothing = words_tag(data_pred_laplaceSmoothing, list_tag)


tags_laplaceSmoothing = correctTagsCounter(tag_sequence_laplaceSmoothing, tag_sequence_gold)
print("Số nhãn đúng là:", tags_laplaceSmoothing, "/", len(tag_sequence_gold), "\n")

tagsList_laplaceSmoothing = {i:tag_sequence_laplaceSmoothing.count(i) for i in tag_sequence_laplaceSmoothing}
print("Tần số xuất hiện của các nhãn khi sử dụng Laplace smoothing:")
print(tagsList_laplaceSmoothing, "\n")

print('Kết quả đánh giá của mô hình HMM khi sử dụng laplace smoothing:\n')
print(classification_report(tag_sequence_gold,tag_sequence_laplaceSmoothing))

Số nhãn đúng là: 52 / 67 

Tần số xuất hiện của các nhãn khi sử dụng Laplace smoothing:
{'N': 35, 'V': 17, 'A': 5, 'P': 6, 'R': 4} 

Kết quả đánh giá của mô hình HMM khi sử dụng laplace smoothing:

              precision    recall  f1-score   support

           A       1.00      0.83      0.91         6
           C       0.00      0.00      0.00         7
           N       0.91      0.94      0.93        34
           P       0.83      1.00      0.91         5
           R       0.25      1.00      0.40         1
           V       0.53      0.64      0.58        14

    accuracy                           0.78        67
   macro avg       0.59      0.74      0.62        67
weighted avg       0.73      0.78      0.75        67

